### Laboratorio 2 - Aprendizaje Supervisado

In [1]:
from utils import load_datasets_unlabeled_test
train, dev, test = load_datasets_unlabeled_test()

In [2]:
import pandas as pd
data_train = pd.DataFrame({'data': train[0], 'target': train[1]})
data_dev = pd.DataFrame({'data': dev[0], 'target': dev[1]})

#### Aumentamos los datos etiquetados utilizando como herramienta traducciones a distintos idiomas

In [3]:
from googletrans import Translator
import pandas as pd
translator = Translator()
text_example = data_train['data'][0].decode('utf-8')
print(text_example + '\n')

translation_es = translator.translate(text_example, dest='es', src='en')
translation_es.text
translation_en = translator.translate(translation_es.text, dest='en', src='es')
print(translation_en.text + '\n')
translation_es = translator.translate(text_example, dest='fr', src='en')
translation_es.text
translation_en = translator.translate(translation_es.text, dest='en', src='fr')
print(translation_en.text + '\n')
translation_es = translator.translate(text_example, dest='it', src='en')
translation_es.text
translation_en = translator.translate(translation_es.text, dest='en', src='it')
print(translation_en.text + '\n')
translation_es = translator.translate(text_example, dest='pt', src='en')
translation_es.text
translation_en = translator.translate(translation_es.text, dest='en', src='pt')
print(translation_en.text + '\n')

If ever a film needed English subtitles this is one . The accents and soft talking are great but hard to follow storyline as you ca n't understand what they are saying and with no subtitles . Her songs were just beautiful and the story is great but a lot of it is lost on not catching what they are saying . But is was a refreshing movie from most out there now . Fine acting and story .

If ever a movie needed subtitles in English, this is one. The accents and the soft conversations are great but difficult to follow because you can not understand what they say and they do not have subtitles. Their songs were simply beautiful and the story is great, but you lose a lot by not grasping what they say. But it was a most refreshing movie now. Good acting and history.

If ever a movie required English subtitles, that's it. Accents and soft conversation are excellent but difficult to follow because you can not understand what they say and without subtitles. Her songs were just beautiful and the 

Como se puede ver, se obtuvieron 4 nuevos datos respecto al original que vamos a asignarle el mismo label (etiqueta) que posee el dato original.

### Unamos los datos de train y dev para aplicar las traducciones a todo el dataset.

In [4]:
print(data_train.shape)
print(data_dev.shape)
data = data_train.append(data_dev, ignore_index=True)
print(data.shape)
data.head()

(963, 2)
(107, 2)
(1070, 2)


,data,target
0,"b""If ever a film needed English subtitles this...",1
1,"b""`` Just Married '' is a painfully cheesy mov...",0
2,"b""I may not be able to add much to the reviews...",1
3,"b""I 've been a 3D nut for many decades . I pre...",0
4,"b""When I found out this version of Lonesome Do...",0


In [5]:
# el metodo translate toma como argumento un string
decoder = lambda x: x.decode('utf-8')
data['data'] = data['data'].apply(decoder)
data.head()

,data,target
0,If ever a film needed English subtitles this i...,1
1,`` Just Married '' is a painfully cheesy movie...,0
2,"I may not be able to add much to the reviews ,...",1
3,I 've been a 3D nut for many decades . I pre-o...,0
4,When I found out this version of Lonesome Dove...,0


In [6]:
from json import JSONDecodeError
# lang shoud be: 'es' 'it' 'pt' 'fr'
def createData(df, lang):
    new_df = pd.DataFrame(columns=['data', 'target'])
    translator = Translator()
    missed_trans = 0
    
    for idx, row in df.iterrows():   
        text = row['data']
        label = row['target']
        if (idx % 10 == 0): print(idx, end=' ')  # kind of 'progress bar' :P
        try:
            translation = translator.translate(text, dest=lang, src='en')
            trans_english = translator.translate(translation.text, dest='en', src=lang)
            # Agrego el nuevo dato al dataframe utilizando la misma etiqueta que el original
            new_df = new_df.append({'data':trans_english.text, 'target':label}, ignore_index=True)
        except JSONDecodeError:
            missed_trans += 1
            
    encoder = lambda x: x.encode('utf-8')
    new_df['data'] = new_df['data'].apply(encoder)
    return new_df, missed_trans

Las siguientes celdas estan comentadas para que no se ejecuten ya que toma demasiado tiempo las traducciones.

In [7]:
# df_es, missed_es = createData(data, 'es')
# print(df_es.shape)
# print(missed_es)
# # Validacion
# print(data.shape[0] == df_es.shape[0] + missed_es)
# # Guardamos el nuevo dataset
# df_es.to_csv('data_from_es_translation.csv', encoding='utf-8', index=False)

In [8]:
# df_it, missed_it = createData(data, 'it')
# print(df_it.shape)
# print(missed_it)
# df_it.to_csv('data_from_it_translation.csv', encoding='utf-8', index=False)

In [9]:
# df_pt, missed_pt = createData(data, 'pt')
# print(df_pt.shape)
# print(missed_pt)
# df_pt.to_csv('data_from_pt_translation.csv', encoding='utf-8', index=False)

In [10]:
# df_fr, missed_fr = createData(data, 'fr')
# print(df_fr.shape)
# print(missed_fr)
# df_fr.to_csv('data_from_fr_translation.csv', encoding='utf-8', index=False)

In [11]:
encoder = lambda x: x.encode('utf-8')
data['data'] = data['data'].apply(encoder)

### Cargamos los nuevos datasets, juntamos todo, aplicamos shuffle y guardamos

In [12]:
df_es = pd.read_csv('data_from_es_translation.csv')
df_it = pd.read_csv('data_from_it_translation.csv')
df_pt = pd.read_csv('data_from_pt_translation.csv')
df_fr = pd.read_csv('data_from_fr_translation.csv')

In [13]:
print(data.shape)
print(df_es.shape)
print(df_it.shape)
print(df_pt.shape)
print(df_fr.shape)
new_data = data.append([df_es, df_it, df_pt, df_fr], ignore_index=True)
print('new data shape:', new_data.shape)
new_data = new_data.sample(frac=1, random_state=0).reset_index(drop=True)
new_data.to_csv('new_data.csv', encoding='utf-8', index=False)

(1070, 2)
(904, 2)
(1045, 2)
(1047, 2)
(1045, 2)
new data shape: (5111, 2)


In [14]:
new_data = pd.read_csv('new_data.csv')
new_data.head()

,data,target
0,"b""The third installment of the popular & quot ...",0
1,"b""The Abyss ( Special Edition ) is not the set...",0
2,"b'I got this DVD for my BDay, and being a big ...",0
3,"b""Although the story surrounds the Spanish Jes...",1
4,b'I was very disappointed when I saw that what...,0


In [15]:
new_data.describe()

,target
count,5111.000000
mean,0.496380
std,0.500036
min,0.000000
25%,0.000000
50%,0.000000
75%,1.000000
max,1.000000


### Dividimos los datos etiquetados en Train y Dev (90% - 10%)

In [16]:
from sklearn.model_selection import train_test_split
X_train, X_dev, y_train, y_dev = train_test_split(new_data.data, new_data.target, 
                                                  test_size=0.10, random_state=42)
print('#train:', X_train.shape[0])
print('#dev:', X_dev.shape[0])
print('#test:', len(test))

#train: 4599
#dev: 512
#test: 500


## CountVectorizer +  LinearSVC:

In [17]:
# El argumento del vectorizador debe ser una lista
X_train, y_train = X_train.tolist(), y_train.tolist()
X_dev, y_dev = X_dev.tolist(), y_dev.tolist()

In [18]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.svm import LinearSVC
from sklearn.pipeline import Pipeline

pipeline = Pipeline([
    ('vect', CountVectorizer()),
    ('clf', LinearSVC(random_state=0)),
])
pipeline.fit(X_train, y_train)

Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip..., max_iter=1000,
     multi_class='ovr', penalty='l2', random_state=0, tol=0.0001,
     verbose=0))])

In [19]:
from utils import print_short_eval, print_eval
print_short_eval(pipeline, X_train, y_train)

accuracy	1.00	macro f1	1.00


In [20]:
print_short_eval(pipeline, X_dev, y_dev)

accuracy	0.99	macro f1	0.99


### Experimento: Binarizar Conteos

In [21]:
pipeline = Pipeline([
    ('vect', CountVectorizer(binary=True)),
    ('clf', LinearSVC(random_state=0)),
])

pipeline.fit(X_train, y_train)

Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=True, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip_..., max_iter=1000,
     multi_class='ovr', penalty='l2', random_state=0, tol=0.0001,
     verbose=0))])

In [22]:
print_short_eval(pipeline, X_train, y_train)

accuracy	1.00	macro f1	1.00


In [23]:
print_short_eval(pipeline, X_dev, y_dev)

accuracy	1.00	macro f1	1.00


#### Conclusión: Binarizar los conteos es una buena elección ya que la accuracy en dev aumenta.

## Distintos Modelos de Clasificación

Probamos distintos modelos de clasificación usando los valores por defecto.

Evaluamos en train (bias) y en dev (variance).

In [22]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC, SVC
from sklearn.ensemble import RandomForestClassifier

clfs = [
    KNeighborsClassifier(),
    MultinomialNB(),
    DecisionTreeClassifier(random_state=0),
    LogisticRegression(random_state=0),
    LinearSVC(random_state=0),
    SVC(random_state=0),
    RandomForestClassifier(random_state=0),
]

In [23]:
vect = CountVectorizer(binary=True)

for clf in clfs:
    print(str(clf.__class__))
    pipeline = Pipeline([
        ('vect', vect),
        ('clf', clf),
    ])
    pipeline.fit(X_train, y_train)
    print_short_eval(pipeline, X_train, y_train)
    print_short_eval(pipeline, X_dev, y_dev)

<class 'sklearn.neighbors.classification.KNeighborsClassifier'>
accuracy	1.00	macro f1	1.00
accuracy	0.98	macro f1	0.98
<class 'sklearn.naive_bayes.MultinomialNB'>
accuracy	0.97	macro f1	0.97
accuracy	0.95	macro f1	0.95
<class 'sklearn.tree.tree.DecisionTreeClassifier'>
accuracy	1.00	macro f1	1.00
accuracy	0.93	macro f1	0.93
<class 'sklearn.linear_model.logistic.LogisticRegression'>
accuracy	1.00	macro f1	1.00
accuracy	0.99	macro f1	0.99
<class 'sklearn.svm.classes.LinearSVC'>
accuracy	1.00	macro f1	1.00
accuracy	1.00	macro f1	1.00
<class 'sklearn.svm.classes.SVC'>
accuracy	0.56	macro f1	0.46
accuracy	0.55	macro f1	0.43
<class 'sklearn.ensemble.forest.RandomForestClassifier'>
accuracy	1.00	macro f1	1.00
accuracy	0.98	macro f1	0.98


## Ahora probamos con el Vectorizador TF-IDF

In [24]:
from sklearn.feature_extraction.text import TfidfVectorizer

vect = TfidfVectorizer(binary=True)
for clf in clfs:
    print(str(clf.__class__))
    pipeline = Pipeline([
        ('vect', vect),
        ('clf', clf),
    ])
    pipeline.fit(X_train, y_train)
    print_short_eval(pipeline, X_train, y_train)
    print_short_eval(pipeline, X_dev, y_dev)

<class 'sklearn.neighbors.classification.KNeighborsClassifier'>
accuracy	1.00	macro f1	1.00
accuracy	0.99	macro f1	0.99
<class 'sklearn.naive_bayes.MultinomialNB'>
accuracy	0.98	macro f1	0.98
accuracy	0.97	macro f1	0.97
<class 'sklearn.tree.tree.DecisionTreeClassifier'>
accuracy	1.00	macro f1	1.00
accuracy	0.89	macro f1	0.89
<class 'sklearn.linear_model.logistic.LogisticRegression'>
accuracy	1.00	macro f1	1.00
accuracy	0.98	macro f1	0.98
<class 'sklearn.svm.classes.LinearSVC'>
accuracy	1.00	macro f1	1.00
accuracy	1.00	macro f1	1.00
<class 'sklearn.svm.classes.SVC'>


/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


accuracy	0.50	macro f1	0.33
accuracy	0.51	macro f1	0.34
<class 'sklearn.ensemble.forest.RandomForestClassifier'>
accuracy	1.00	macro f1	1.00
accuracy	0.97	macro f1	0.97


### Conclusiones:
- Todos los modelos anteriores, excepto SVC y DecisionTreeClassifier, son prometedores.
- En general, tienen mejor accuracy si se utiliza CountVectorizer.

### IDEA: 
Ajustar hiperparámetros en los modelos más prometedores y luego utilizar VotingClassifier utilizando la mejor versión de cada uno de ellos.

Primero probemos VotingClassifier sin ajustar hiperparámetros

In [27]:
from sklearn.ensemble import VotingClassifier

vect = CountVectorizer(binary=False)
clf1 = Pipeline([
    ('vect', vect),
    ('clf', KNeighborsClassifier()),
])
clf2 = Pipeline([
    ('vect', vect),
    ('clf', MultinomialNB()),
])
clf3 = Pipeline([
    ('vect', vect),
    ('clf', LogisticRegression(random_state=0)),
])
clf4 = Pipeline([
    ('vect', vect),
    ('clf', LinearSVC(random_state=0)),
])
clf5 = Pipeline([
    ('vect', vect),
    ('clf', RandomForestClassifier(random_state=0)),
])

eclf1 = VotingClassifier(estimators=[('knc', clf1), ('mnb', clf2), ('lr', clf3),
                                     ('lsvc', clf4), ('rfc', clf5)], voting='hard')
eclf1 = eclf1.fit(X_train, y_train)
print_eval(eclf1, X_train, y_train)
print_eval(eclf1, X_dev, y_dev)

/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


accuracy	1.00

             precision    recall  f1-score   support

        neg       1.00      1.00      1.00      2311
        pos       1.00      1.00      1.00      2288

avg / total       1.00      1.00      1.00      4599

[[2311    0]
 [   0 2288]]
accuracy	1.00

             precision    recall  f1-score   support

        neg       1.00      1.00      1.00       263
        pos       1.00      1.00      1.00       249

avg / total       1.00      1.00      1.00       512

[[262   1]
 [  0 249]]


/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


Excelentes resultados! 

In [28]:
# UNIMOS train y dev:
train = X_train + X_dev
target = y_train + y_dev

In [29]:
vect = CountVectorizer(binary=True)
clf1 = Pipeline([
    ('vect', vect),
    ('clf', KNeighborsClassifier()),
])
clf2 = Pipeline([
    ('vect', vect),
    ('clf', MultinomialNB()),
])
clf3 = Pipeline([
    ('vect', vect),
    ('clf', LogisticRegression(random_state=0)),
])
clf4 = Pipeline([
    ('vect', vect),
    ('clf', LinearSVC(random_state=0)),
])
clf5 = Pipeline([
    ('vect', vect),
    ('clf', RandomForestClassifier(random_state=0)),
])

eclf1 = VotingClassifier(estimators=[('knc', clf1), ('mnb', clf2), ('lr', clf3),
                                     ('lsvc', clf4), ('rfc', clf5)], voting='hard')
eclf1 = eclf1.fit(train, target)
print_eval(eclf1, train, target)

accuracy	1.00

             precision    recall  f1-score   support

        neg       1.00      1.00      1.00      2574
        pos       1.00      1.00      1.00      2537

avg / total       1.00      1.00      1.00      5111

[[2574    0]
 [   0 2537]]


/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


In [118]:
y_pred = eclf1.predict(test)
predictions = y_pred.tolist()
from utils import save_results
save_results('results_moreData_Voting.csv', predictions)

/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


Se obtuvo un accuracy del 88%

# Pasos a seguir

Con el afán de encontrar un modelo que generalice lo mejor posible (i.e. sea capaz de clasificar bien datos nuevos), vamos a ajustar algunos hiperparámetros de cada modelo por separado.

### Intentemos mejorar LogisticRegression (acc dev actual = 99%)

In [31]:
pipeline = Pipeline([
    ('vect', CountVectorizer(binary=True)),
    ('clf', LogisticRegression(random_state=0)),
])
pipeline.fit(X_train, y_train)
print_eval(pipeline, X_train, y_train)
print_eval(pipeline, X_dev, y_dev)

accuracy	1.00

             precision    recall  f1-score   support

        neg       1.00      1.00      1.00      2311
        pos       1.00      1.00      1.00      2288

avg / total       1.00      1.00      1.00      4599

[[2311    0]
 [   0 2288]]
accuracy	0.99

             precision    recall  f1-score   support

        neg       1.00      0.99      0.99       263
        pos       0.99      1.00      0.99       249

avg / total       0.99      0.99      0.99       512

[[261   2]
 [  1 248]]


### Grid-Search en Dev

In [32]:
from sklearn.model_selection import ParameterGrid

param_grid = {
    'vect__binary': [True],
    'vect__ngram_range': [(1, 1), (1, 2), (1, 3), (1, 4), (1, 5)],
#    'vect__min_df': [1, 3, 5, 7],
#    'vect__max_df': [0.95, 0.9, 0.7],
    'clf__random_state': [0],
    # parameter for LogisticRegression (smaller values -> stronger regularization)
    'clf__C': [0.001, 0.005, 0.01, 0.05, 0.1, 0.3, 0.5, 1.0]
}

params_list = list(ParameterGrid(param_grid))
len(params_list)

40

In [110]:
from utils import eval

pipeline = Pipeline([
    ('vect', CountVectorizer()),
    ('clf', LogisticRegression()),
])
results = []
for params in params_list:
    pipeline.set_params(**params)
    pipeline.fit(X_train, y_train)
    result = eval(pipeline, X_dev, y_dev)
#     print(result)
    results.append({
        **result,
        **params,
    })

{'acc': 0.87109375, 'f1': 0.8708553084155011}
{'acc': 0.927734375, 'f1': 0.9275616685594545}
{'acc': 0.947265625, 'f1': 0.9471395959758182}
{'acc': 0.951171875, 'f1': 0.9510357095040454}
{'acc': 0.95703125, 'f1': 0.9569023569023569}
{'acc': 0.912109375, 'f1': 0.9119881734108021}
{'acc': 0.95703125, 'f1': 0.9569366283337413}
{'acc': 0.974609375, 'f1': 0.9745665888430779}
{'acc': 0.98828125, 'f1': 0.9882697947214076}
{'acc': 0.990234375, 'f1': 0.9902259858964031}
{'acc': 0.9375, 'f1': 0.9374226567871056}
{'acc': 0.978515625, 'f1': 0.9784919134635581}
{'acc': 0.9921875, 'f1': 0.9921798631476051}
{'acc': 0.9921875, 'f1': 0.9921798631476051}
{'acc': 0.994140625, 'f1': 0.9941355915378418}
{'acc': 0.97265625, 'f1': 0.9726224123443588}
{'acc': 0.998046875, 'f1': 0.9980456150396031}
{'acc': 0.998046875, 'f1': 0.9980456150396031}
{'acc': 0.998046875, 'f1': 0.9980456150396031}
{'acc': 0.998046875, 'f1': 0.9980456150396031}
{'acc': 0.978515625, 'f1': 0.9784919134635581}
{'acc': 0.998046875, 'f1': 

In [112]:
results_df = pd.DataFrame(results)
results_df.sort_values(['acc', 'f1'], ascending=False)[:10]

,acc,clf__C,clf__random_state,f1,vect__binary,vect__ngram_range
16,0.998047,0.05,0,0.998046,True,"(1, 2)"
17,0.998047,0.05,0,0.998046,True,"(1, 3)"
18,0.998047,0.05,0,0.998046,True,"(1, 4)"
19,0.998047,0.05,0,0.998046,True,"(1, 5)"
21,0.998047,0.10,0,0.998046,True,"(1, 2)"
22,0.998047,0.10,0,0.998046,True,"(1, 3)"
23,0.998047,0.10,0,0.998046,True,"(1, 4)"
24,0.998047,0.10,0,0.998046,True,"(1, 5)"
26,0.998047,0.30,0,0.998046,True,"(1, 2)"
27,0.998047,0.30,0,0.998046,True,"(1, 3)"


Entrenamos nuestro modelo con los mejores hiperparámetros obtenidos y luego evaluamos

In [33]:
pipeline = Pipeline([
    ('vect', CountVectorizer(binary=True, ngram_range=(1,2))),
    ('clf', LogisticRegression(random_state=0, C=0.05)),
])
pipeline.fit(X_train, y_train)
print_eval(pipeline, X_train, y_train)
print_eval(pipeline, X_dev, y_dev)

accuracy	1.00

             precision    recall  f1-score   support

        neg       1.00      1.00      1.00      2311
        pos       1.00      1.00      1.00      2288

avg / total       1.00      1.00      1.00      4599

[[2311    0]
 [   1 2287]]
accuracy	1.00

             precision    recall  f1-score   support

        neg       1.00      1.00      1.00       263
        pos       1.00      1.00      1.00       249

avg / total       1.00      1.00      1.00       512

[[262   1]
 [  0 249]]


### Se obtuvo una mejora de performance! (de 99% a ~100% en dev, se equivoca en un ejemplo)

## Intentemos mejorar MultinomialNB (acc dev actual = 95%)

In [35]:
pipeline = Pipeline([
    ('vect', CountVectorizer(binary=True)),
    ('clf', MultinomialNB()),
])
pipeline.fit(X_train, y_train)
print_short_eval(pipeline, X_train, y_train)
print_short_eval(pipeline, X_dev, y_dev)

accuracy	0.97	macro f1	0.97
accuracy	0.95	macro f1	0.95


### Grid-Search en Dev

In [162]:
param_grid = {
    'vect__binary': [True],
    'vect__ngram_range': [(1, 1)],
    'vect__min_df': [3, 5],
    'vect__max_df': [0.95, 0.9, 0.7],
    'clf__alpha': [0.1, 0.5, 1.0, 5.0, 8.0, 10.0, 15.0, 20.0]
}

params_list = list(ParameterGrid(param_grid))
len(params_list)

48

In [163]:
pipeline = Pipeline([
    ('vect', CountVectorizer()),
    ('clf', MultinomialNB()),
])
# pipeline.get_params().keys()
results = []
for params in params_list:
    pipeline.set_params(**params)
    pipeline.fit(X_train, y_train)
    result = eval(pipeline, X_dev, y_dev)
#    print(result)
    results.append({
        **result,
        **params,
    })

In [164]:
results_df = pd.DataFrame(results)
results_df.sort_values(['acc', 'f1'], ascending=False)[:10]

,acc,clf__alpha,f1,vect__binary,vect__max_df,vect__min_df,vect__ngram_range
4,0.970703,0.1,0.970621,True,0.70,3,"(1, 1)"
0,0.968750,0.1,0.968656,True,0.95,3,"(1, 1)"
2,0.968750,0.1,0.968656,True,0.90,3,"(1, 1)"
10,0.966797,0.5,0.966690,True,0.70,3,"(1, 1)"
5,0.960938,0.1,0.960820,True,0.70,5,"(1, 1)"
16,0.960938,1.0,0.960784,True,0.70,3,"(1, 1)"
1,0.958984,0.1,0.958833,True,0.95,5,"(1, 1)"
3,0.958984,0.1,0.958833,True,0.90,5,"(1, 1)"
6,0.958984,0.5,0.958813,True,0.95,3,"(1, 1)"
8,0.958984,0.5,0.958813,True,0.90,3,"(1, 1)"


Entrenamos nuestro modelo con los mejores hiperparámetros obtenidos y luego evaluamos

In [36]:
pipeline = Pipeline([
    ('vect', CountVectorizer(binary=True, max_df=0.70, min_df=3, ngram_range=(1,1))),
    ('clf', MultinomialNB(alpha=0.1)),
])
pipeline.fit(X_train, y_train)
print_eval(pipeline, X_train, y_train)
print_eval(pipeline, X_dev, y_dev)

accuracy	0.98

             precision    recall  f1-score   support

        neg       0.97      1.00      0.98      2311
        pos       1.00      0.97      0.98      2288

avg / total       0.98      0.98      0.98      4599

[[2311    0]
 [  76 2212]]
accuracy	0.97

             precision    recall  f1-score   support

        neg       0.95      1.00      0.97       263
        pos       1.00      0.94      0.97       249

avg / total       0.97      0.97      0.97       512

[[262   1]
 [ 14 235]]


### Se obtuvo una mejora de performance! (de 95% a 97% en dev)

## Intentemos mejorar KNeighborsClassifier (acc dev actual = 98%)

In [37]:
pipeline = Pipeline([
    ('vect', CountVectorizer(binary=True)),
    ('clf', KNeighborsClassifier()),
])
pipeline.fit(X_train, y_train)
print_short_eval(pipeline, X_train, y_train)
print_short_eval(pipeline, X_dev, y_dev)

accuracy	1.00	macro f1	1.00
accuracy	0.98	macro f1	0.98


### Grid-Search in Dev

In [38]:
from sklearn.model_selection import ParameterGrid

param_grid = {
    'vect__binary': [True],
    'vect__ngram_range': [(1, 1)],
    'vect__min_df': [1, 3, 5],
    'vect__max_df': [0.95, 0.9, 0.7],
    'clf__n_neighbors': [1, 2, 5, 8, 10, 20, 30, 50, 100]
}

params_list = list(ParameterGrid(param_grid))
len(params_list)

81

In [169]:
pipeline = Pipeline([
    ('vect', CountVectorizer()),
    ('clf', KNeighborsClassifier()),
])
# pipeline.get_params().keys()
results = []
for params in params_list:
    pipeline.set_params(**params)
    pipeline.fit(X_train, y_train)
    result = eval(pipeline, X_dev, y_dev)
# print(result)
    results.append({
        **result,
        **params,
    })

In [172]:
results_df = pd.DataFrame(results)
results_df.sort_values(['acc', 'f1'], ascending=False)[:10]

,acc,clf__n_neighbors,f1,vect__binary,vect__max_df,vect__min_df,vect__ngram_range
6,1.000000,1,1.000000,True,0.70,1,"(1, 1)"
7,1.000000,1,1.000000,True,0.70,3,"(1, 1)"
0,0.998047,1,0.998045,True,0.95,1,"(1, 1)"
1,0.998047,1,0.998045,True,0.95,3,"(1, 1)"
2,0.998047,1,0.998045,True,0.95,5,"(1, 1)"
3,0.998047,1,0.998045,True,0.90,1,"(1, 1)"
4,0.998047,1,0.998045,True,0.90,3,"(1, 1)"
5,0.998047,1,0.998045,True,0.90,5,"(1, 1)"
8,0.998047,1,0.998045,True,0.70,5,"(1, 1)"
15,0.996094,2,0.996090,True,0.70,1,"(1, 1)"


Entrenamos nuestro modelo con los mejores hiperparámetros obtenidos y luego evaluamos

In [40]:
pipeline = Pipeline([
    ('vect', CountVectorizer(binary=True, max_df=0.70, min_df=1, ngram_range=(1,1))),
    ('clf', KNeighborsClassifier(n_neighbors=1))
])
pipeline.fit(X_train, y_train)
print_eval(pipeline, X_train, y_train)
print_eval(pipeline, X_dev, y_dev)

accuracy	1.00

             precision    recall  f1-score   support

        neg       1.00      1.00      1.00      2311
        pos       1.00      1.00      1.00      2288

avg / total       1.00      1.00      1.00      4599

[[2311    0]
 [   0 2288]]
accuracy	1.00

             precision    recall  f1-score   support

        neg       1.00      1.00      1.00       263
        pos       1.00      1.00      1.00       249

avg / total       1.00      1.00      1.00       512

[[263   0]
 [  0 249]]


### Se obtuvo una mejora de performance! (de 98% a 100% en dev)

## Intentemos mejorar LinearSVC

In [174]:
pipeline = Pipeline([
    ('vect', CountVectorizer(binary=True)),
    ('clf', LinearSVC(random_state=0)),
])
pipeline.fit(X_train, y_train)
print_eval(pipeline, X_train, y_train)
print_eval(pipeline, X_dev, y_dev)

accuracy	1.00

             precision    recall  f1-score   support

        neg       1.00      1.00      1.00      2311
        pos       1.00      1.00      1.00      2288

avg / total       1.00      1.00      1.00      4599

[[2311    0]
 [   0 2288]]
accuracy	1.00

             precision    recall  f1-score   support

        neg       1.00      1.00      1.00       263
        pos       1.00      1.00      1.00       249

avg / total       1.00      1.00      1.00       512

[[262   1]
 [  1 248]]


### Grid-Search in Dev

In [175]:
param_grid = {
    'vect__binary': [True],
    'vect__ngram_range': [(1,3), (1,4), (1,5)],
    'vect__min_df': [5],
    'vect__max_df': [0.95],
    'clf__random_state': [0],
    'clf__C': [0.001, 0.01, 0.05, 0.1, 0.3, 0.5, 1.0, 2.0, 5.0, 10.0]
}

params_list = list(ParameterGrid(param_grid))
len(params_list)

30

In [176]:
pipeline = Pipeline([
    ('vect', CountVectorizer()),
    ('clf', LinearSVC()),
])
# pipeline.get_params().keys()
results = []
for params in params_list:
    pipeline.set_params(**params)
    pipeline.fit(X_train, y_train)
    result = eval(pipeline, X_dev, y_dev)
#     print(result)
    results.append({
        **result,
        **params,
    })

{'acc': 0.978515625, 'f1': 0.9784919134635581}
{'acc': 0.98046875, 'f1': 0.9804496578690127}
{'acc': 0.98046875, 'f1': 0.9804496578690127}
{'acc': 0.998046875, 'f1': 0.9980456150396031}
{'acc': 0.998046875, 'f1': 0.9980456150396031}
{'acc': 0.998046875, 'f1': 0.9980456150396031}
{'acc': 0.998046875, 'f1': 0.9980456150396031}
{'acc': 0.998046875, 'f1': 0.9980456150396031}
{'acc': 0.998046875, 'f1': 0.9980456150396031}
{'acc': 0.998046875, 'f1': 0.9980456150396031}
{'acc': 0.998046875, 'f1': 0.9980456150396031}
{'acc': 0.998046875, 'f1': 0.9980456150396031}
{'acc': 0.998046875, 'f1': 0.9980456150396031}
{'acc': 0.998046875, 'f1': 0.9980456150396031}
{'acc': 0.998046875, 'f1': 0.9980456150396031}
{'acc': 0.998046875, 'f1': 0.9980456150396031}
{'acc': 0.998046875, 'f1': 0.9980456150396031}
{'acc': 0.998046875, 'f1': 0.9980456150396031}
{'acc': 0.998046875, 'f1': 0.9980456150396031}
{'acc': 0.998046875, 'f1': 0.9980456150396031}
{'acc': 0.998046875, 'f1': 0.9980456150396031}
{'acc': 0.99804

In [177]:
results_df = pd.DataFrame(results)
results_df.sort_values(['acc', 'f1'], ascending=False)[:10]

,acc,clf__C,clf__random_state,f1,vect__binary,vect__max_df,vect__min_df,vect__ngram_range
3,0.998047,0.01,0,0.998046,True,0.95,5,"(1, 3)"
4,0.998047,0.01,0,0.998046,True,0.95,5,"(1, 4)"
5,0.998047,0.01,0,0.998046,True,0.95,5,"(1, 5)"
6,0.998047,0.05,0,0.998046,True,0.95,5,"(1, 3)"
7,0.998047,0.05,0,0.998046,True,0.95,5,"(1, 4)"
8,0.998047,0.05,0,0.998046,True,0.95,5,"(1, 5)"
9,0.998047,0.10,0,0.998046,True,0.95,5,"(1, 3)"
10,0.998047,0.10,0,0.998046,True,0.95,5,"(1, 4)"
11,0.998047,0.10,0,0.998046,True,0.95,5,"(1, 5)"
12,0.998047,0.30,0,0.998046,True,0.95,5,"(1, 3)"


Entrenamos nuestro modelo con los mejores hiperparámetros obtenidos y luego evaluamos

In [41]:
pipeline = Pipeline([
    ('vect', CountVectorizer(binary=True, max_df=0.95, min_df=5, ngram_range=(1,3))),
    ('clf', LinearSVC(random_state=0, C=0.01))
])
pipeline.fit(X_train, y_train)
print_eval(pipeline, X_train, y_train)
print_eval(pipeline, X_dev, y_dev)

accuracy	1.00

             precision    recall  f1-score   support

        neg       1.00      1.00      1.00      2311
        pos       1.00      1.00      1.00      2288

avg / total       1.00      1.00      1.00      4599

[[2311    0]
 [   0 2288]]
accuracy	1.00

             precision    recall  f1-score   support

        neg       1.00      1.00      1.00       263
        pos       1.00      1.00      1.00       249

avg / total       1.00      1.00      1.00       512

[[262   1]
 [  0 249]]


### Se obtuvo una pequeña mejora, pudo etiquetar 1 ejemplo más de manera correcta

In [179]:
# UNIMOS train y dev:
train = X_train + X_dev
target = y_train + y_dev

In [181]:
pipeline = Pipeline([
    ('vect', CountVectorizer(binary=True, max_df=0.95, min_df=5, ngram_range=(1,3))),
    ('clf', LinearSVC(random_state=0, C=0.01))
])
pipeline = pipeline.fit(train, target)
print_eval(pipeline, train, target)

accuracy	1.00

             precision    recall  f1-score   support

        neg       1.00      1.00      1.00      2574
        pos       1.00      1.00      1.00      2537

avg / total       1.00      1.00      1.00      5111

[[2574    0]
 [   0 2537]]


In [182]:
y_pred = pipeline.predict(test)
predictions = y_pred.tolist()
from utils import save_results
save_results('results_moreData_LinearSVC.csv', predictions)

Se obtuvo un accuracy del 88%

## Intentemos mejorar RandomForest (acc actual en dev = 98%)

In [42]:
pipeline = Pipeline([
    ('vect', CountVectorizer(binary=True)),
    ('clf', RandomForestClassifier(random_state=0)),
])
pipeline.fit(X_train, y_train)
print_eval(pipeline, X_train, y_train)
print_eval(pipeline, X_dev, y_dev)

accuracy	1.00

             precision    recall  f1-score   support

        neg       1.00      1.00      1.00      2311
        pos       1.00      1.00      1.00      2288

avg / total       1.00      1.00      1.00      4599

[[2311    0]
 [   1 2287]]
accuracy	0.98

             precision    recall  f1-score   support

        neg       0.97      1.00      0.98       263
        pos       1.00      0.96      0.98       249

avg / total       0.98      0.98      0.98       512

[[262   1]
 [  9 240]]


### Grid-Search in Dev

In [195]:
param_grid = {
    'vect__binary': [True],
    'vect__ngram_range': [(1,1), (1,3)],
    'vect__min_df': [3],
    'vect__max_df': [0.95],
    'clf__random_state': [0],
    'clf__n_estimators': [5, 10, 20, 50, 100],
    'clf__criterion': ['gini', 'entropy']
}

params_list = list(ParameterGrid(param_grid))
len(params_list)

20

In [196]:
pipeline = Pipeline([
    ('vect', CountVectorizer()),
    ('clf', RandomForestClassifier()),
])
# pipeline.get_params().keys()
results = []
for params in params_list:
    pipeline.set_params(**params)
    pipeline.fit(X_train, y_train)
    result = eval(pipeline, X_dev, y_dev)
#     print(result)
    results.append({
        **result,
        **params,
    })

{'acc': 0.96875, 'f1': 0.9687266174966025}
{'acc': 0.951171875, 'f1': 0.9511179851444501}
{'acc': 0.994140625, 'f1': 0.9941341582173341}
{'acc': 0.984375, 'f1': 0.9843511217067058}
{'acc': 0.99609375, 'f1': 0.9960899315738025}
{'acc': 0.994140625, 'f1': 0.9941355915378418}
{'acc': 1.0, 'f1': 1.0}
{'acc': 0.998046875, 'f1': 0.9980456150396031}
{'acc': 0.998046875, 'f1': 0.9980456150396031}
{'acc': 0.998046875, 'f1': 0.9980456150396031}
{'acc': 0.966796875, 'f1': 0.9667683520477705}
{'acc': 0.98046875, 'f1': 0.9804612965761474}
{'acc': 0.982421875, 'f1': 0.982402474652002}
{'acc': 0.98828125, 'f1': 0.9882667481475822}
{'acc': 0.994140625, 'f1': 0.9941368451188091}
{'acc': 0.99609375, 'f1': 0.9960899315738025}
{'acc': 0.998046875, 'f1': 0.9980456150396031}
{'acc': 0.998046875, 'f1': 0.9980456150396031}
{'acc': 0.998046875, 'f1': 0.9980456150396031}
{'acc': 0.998046875, 'f1': 0.9980456150396031}


In [197]:
results_df = pd.DataFrame(results)
results_df.sort_values(['acc', 'f1'], ascending=False)[:10]

,acc,clf__criterion,clf__n_estimators,clf__random_state,f1,vect__binary,vect__max_df,vect__min_df,vect__ngram_range
6,1.000000,gini,50,0,1.000000,True,0.95,3,"(1, 1)"
7,0.998047,gini,50,0,0.998046,True,0.95,3,"(1, 3)"
8,0.998047,gini,100,0,0.998046,True,0.95,3,"(1, 1)"
9,0.998047,gini,100,0,0.998046,True,0.95,3,"(1, 3)"
16,0.998047,entropy,50,0,0.998046,True,0.95,3,"(1, 1)"
17,0.998047,entropy,50,0,0.998046,True,0.95,3,"(1, 3)"
18,0.998047,entropy,100,0,0.998046,True,0.95,3,"(1, 1)"
19,0.998047,entropy,100,0,0.998046,True,0.95,3,"(1, 3)"
4,0.996094,gini,20,0,0.996090,True,0.95,3,"(1, 1)"
15,0.996094,entropy,20,0,0.996090,True,0.95,3,"(1, 3)"


Entrenamos nuestro modelo con los mejores hiperparámetros obtenidos y luego evaluamos

In [43]:
pipeline = Pipeline([
    ('vect', CountVectorizer(binary=True, max_df=0.95, min_df=3, ngram_range=(1,1))),
    ('clf', RandomForestClassifier(random_state=0, n_estimators=50, criterion='gini'))
])
pipeline.fit(X_train, y_train)
print_eval(pipeline, X_train, y_train)
print_eval(pipeline, X_dev, y_dev)

accuracy	1.00

             precision    recall  f1-score   support

        neg       1.00      1.00      1.00      2311
        pos       1.00      1.00      1.00      2288

avg / total       1.00      1.00      1.00      4599

[[2311    0]
 [   0 2288]]
accuracy	1.00

             precision    recall  f1-score   support

        neg       1.00      1.00      1.00       263
        pos       1.00      1.00      1.00       249

avg / total       1.00      1.00      1.00       512

[[263   0]
 [  0 249]]


### Gran mejora en RandomForest! El modelo tiene bias y varianza igual a 0

In [199]:
# UNIMOS train y dev:
train = X_train + X_dev
target = y_train + y_dev

In [201]:
pipeline = Pipeline([
    ('vect', CountVectorizer(binary=True, max_df=0.95, min_df=3, ngram_range=(1,1))),
    ('clf', RandomForestClassifier(random_state=0, n_estimators=50, criterion='gini'))
])
pipeline = pipeline.fit(train, target)
print_eval(pipeline, train, target)

accuracy	1.00

             precision    recall  f1-score   support

        neg       1.00      1.00      1.00      2574
        pos       1.00      1.00      1.00      2537

avg / total       1.00      1.00      1.00      5111

[[2574    0]
 [   0 2537]]


In [202]:
y_pred = pipeline.predict(test)
predictions = y_pred.tolist()
from utils import save_results
save_results('results_moreData_RandomForest.csv', predictions)

Se obtuvo un accuracy del 80%

### Ahora hacemos VotingClassifier con las mejores versiones de los modelos

In [44]:
clf1 = Pipeline([
    ('vect', CountVectorizer(binary=True, max_df=0.70, min_df=1, ngram_range=(1,1))),
    ('clf1', KNeighborsClassifier(n_neighbors=1))
])
clf2 = Pipeline([
    ('vect', CountVectorizer(binary=True, max_df=0.70, min_df=3, ngram_range=(1,1))),
    ('clf2', MultinomialNB(alpha=0.1)),
])
clf3 = Pipeline([
    ('vect', CountVectorizer(binary=True, ngram_range=(1,2))),
    ('clf3', LogisticRegression(random_state=0, C=0.05)),
])
clf4 = Pipeline([
    ('vect', CountVectorizer(binary=True, max_df=0.95, min_df=5, ngram_range=(1,3))),
    ('clf4', LinearSVC(random_state=0, C=0.01))
])
clf5 = Pipeline([
    ('vect', CountVectorizer(binary=True, max_df=0.95, min_df=3, ngram_range=(1,1))),
    ('clf5', RandomForestClassifier(random_state=0, n_estimators=50, criterion='gini'))
])

eclf1 = VotingClassifier(estimators=[('knc', clf1), ('mnb', clf2), ('lr', clf3),
                                     ('lsvc', clf4), ('rfc', clf5)], voting='hard')
eclf1 = eclf1.fit(X_train, y_train)
print_eval(eclf1, X_train, y_train)
print_eval(eclf1, X_dev, y_dev)

/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


accuracy	1.00

             precision    recall  f1-score   support

        neg       1.00      1.00      1.00      2311
        pos       1.00      1.00      1.00      2288

avg / total       1.00      1.00      1.00      4599

[[2311    0]
 [   0 2288]]
accuracy	1.00

             precision    recall  f1-score   support

        neg       1.00      1.00      1.00       263
        pos       1.00      1.00      1.00       249

avg / total       1.00      1.00      1.00       512

[[262   1]
 [  0 249]]


/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


In [45]:
clf1 = Pipeline([
    ('vect', CountVectorizer(binary=True, max_df=0.70, min_df=1, ngram_range=(1,1))),
    ('clf1', KNeighborsClassifier(n_neighbors=1))
])
clf2 = Pipeline([
    ('vect', CountVectorizer(binary=True, max_df=0.70, min_df=3, ngram_range=(1,1))),
    ('clf2', MultinomialNB(alpha=0.1)),
])
clf3 = Pipeline([
    ('vect', CountVectorizer(binary=True, ngram_range=(1,2))),
    ('clf3', LogisticRegression(random_state=0, C=0.05)),
])
clf4 = Pipeline([
    ('vect', CountVectorizer(binary=True, max_df=0.95, min_df=5, ngram_range=(1,3))),
    ('clf4', LinearSVC(random_state=0, C=0.01))
])
clf5 = Pipeline([
    ('vect', CountVectorizer(binary=True, max_df=0.95, min_df=3, ngram_range=(1,1))),
    ('clf5', RandomForestClassifier(random_state=0, n_estimators=50, criterion='gini'))
])

eclf1 = VotingClassifier(estimators=[('knc', clf1), ('mnb', clf2), ('lr', clf3),
                                     ('lsvc', clf4), ('rfc', clf5)], voting='hard')
eclf1 = eclf1.fit(train, target)
print_eval(eclf1, train, target)

accuracy	1.00

             precision    recall  f1-score   support

        neg       1.00      1.00      1.00      2574
        pos       1.00      1.00      1.00      2537

avg / total       1.00      1.00      1.00      5111

[[2574    0]
 [   0 2537]]


/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


In [208]:
y_pred = eclf1.predict(test)
predictions = y_pred.tolist()
from utils import save_results
save_results('results_moreData_finalVoting.csv', predictions)

/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


Se obtuvo una accuracy del 84.66%. 

- No hubo mejoras, de hecho el modelo de Voting anterior que utiliza los modelos sin los ajustes de hiperparametros tiene mejor accuracy en test.

- Probablemente usar VotingClassifier no sea una buena opción para este problema.

- Tal vez se deberia probar dando mas peso a algunos clasificadores frente a otros.

In [57]:
clf1 = Pipeline([
    ('vect', CountVectorizer(binary=True, max_df=0.70, min_df=1, ngram_range=(1,1))),
    ('clf1', KNeighborsClassifier(n_neighbors=1))
])
clf3 = Pipeline([
    ('vect', CountVectorizer(binary=True, ngram_range=(1,2))),
    ('clf3', LogisticRegression(random_state=0, C=0.05)),
])

eclf_soft = VotingClassifier(estimators=[('kn', clf1), ('lr', clf3)], voting='soft')
# eclf_soft = eclf_soft.fit(train, target)
# print_eval(eclf_soft, train, target)

In [60]:
clf4 = Pipeline([
    ('vect', CountVectorizer(binary=True, max_df=0.95, min_df=5, ngram_range=(1,3))),
    ('clf4', LinearSVC(random_state=0, C=0.01))
])
clf5 = Pipeline([
    ('vect', CountVectorizer(binary=True, max_df=0.95, min_df=3, ngram_range=(1,1))),
    ('clf5', RandomForestClassifier(random_state=0, n_estimators=50, criterion='gini'))
])

eclf_hard = VotingClassifier(estimators=[('eclf_soft', eclf_soft), ('lsvc', clf4),
                                         ('rfc', clf5)], voting='hard')
# eclf_hard = eclf_hard.fit(X_train, y_train)
# print_eval(eclf_hard, X_train, y_train)
# print_eval(eclf_hard, X_dev, y_dev)
eclf_hard = eclf_hard.fit(train, target)
print_eval(eclf_hard, train, target)

/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


accuracy	1.00

             precision    recall  f1-score   support

        neg       1.00      1.00      1.00      2574
        pos       1.00      1.00      1.00      2537

avg / total       1.00      1.00      1.00      5111

[[2574    0]
 [   0 2537]]


/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


In [61]:
y_pred = eclf_hard.predict(test)
predictions = y_pred.tolist()
from utils import save_results
save_results('results_moreData_Voting_soft_hard.csv', predictions)

/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


Se obtuvo una accuracy del 82.66% (peor que la del voting anterior 84.66%)

## Intentemos con el modelo "one example": es un vectorizador TFIDF + un clasificador logistic regression CV

In [38]:
from sklearn.linear_model import LogisticRegressionCV
pipeline = Pipeline([
    ('vect', TfidfVectorizer(binary=True)),
    ('clf', LogisticRegressionCV(random_state=0, cv=10)),
])
pipeline.fit(X_train, y_train)
print_eval(pipeline, X_train, y_train)
print_eval(pipeline, X_dev, y_dev)

accuracy	1.00

             precision    recall  f1-score   support

        neg       1.00      1.00      1.00      2311
        pos       1.00      1.00      1.00      2288

avg / total       1.00      1.00      1.00      4599

[[2311    0]
 [   0 2288]]
accuracy	1.00

             precision    recall  f1-score   support

        neg       1.00      1.00      1.00       263
        pos       1.00      1.00      1.00       249

avg / total       1.00      1.00      1.00       512

[[262   1]
 [  0 249]]


### Grid-Search en Dev

In [55]:
from sklearn.model_selection import ParameterGrid

param_grid = {
    'vect__binary': [True],
    'vect__ngram_range': [(1, 1), (1, 2), (1, 3)],
    'vect__min_df': [0, 1, 3, 5],
    'vect__max_df': [1, 0.95, 0.7],
    'clf__random_state': [0],
    # parameter for LogisticRegression (smaller values -> stronger regularization)
    'clf__cv': [1, 2, 5, 10]
}

params_list = list(ParameterGrid(param_grid))
len(params_list)

144

In [56]:
from utils import eval

pipeline = Pipeline([
    ('vect', TfidfVectorizer()),
    ('clf', LogisticRegressionCV()),
])
results = []
for params in params_list:
    pipeline.set_params(**params)
    pipeline.fit(X_train, y_train)
    result = eval(pipeline, X_dev, y_dev)
    print(result)
    results.append({
        **result,
        **params,
    })

ValueError: k-fold cross-validation requires at least one train/test split by setting n_splits=2 or more, got n_splits=1.

In [112]:
results_df = pd.DataFrame(results)
results_df.sort_values(['acc', 'f1'], ascending=False)[:10]

,acc,clf__C,clf__random_state,f1,vect__binary,vect__ngram_range
16,0.998047,0.05,0,0.998046,True,"(1, 2)"
17,0.998047,0.05,0,0.998046,True,"(1, 3)"
18,0.998047,0.05,0,0.998046,True,"(1, 4)"
19,0.998047,0.05,0,0.998046,True,"(1, 5)"
21,0.998047,0.10,0,0.998046,True,"(1, 2)"
22,0.998047,0.10,0,0.998046,True,"(1, 3)"
23,0.998047,0.10,0,0.998046,True,"(1, 4)"
24,0.998047,0.10,0,0.998046,True,"(1, 5)"
26,0.998047,0.30,0,0.998046,True,"(1, 2)"
27,0.998047,0.30,0,0.998046,True,"(1, 3)"


Entrenamos nuestro modelo con los mejores hiperparámetros obtenidos y luego evaluamos

In [33]:
pipeline = Pipeline([
    ('vect', CountVectorizer(binary=True, ngram_range=(1,2))),
    ('clf', LogisticRegression(random_state=0, C=0.05)),
])
pipeline.fit(X_train, y_train)
print_eval(pipeline, X_train, y_train)
print_eval(pipeline, X_dev, y_dev)

accuracy	1.00

             precision    recall  f1-score   support

        neg       1.00      1.00      1.00      2311
        pos       1.00      1.00      1.00      2288

avg / total       1.00      1.00      1.00      4599

[[2311    0]
 [   1 2287]]
accuracy	1.00

             precision    recall  f1-score   support

        neg       1.00      1.00      1.00       263
        pos       1.00      1.00      1.00       249

avg / total       1.00      1.00      1.00       512

[[262   1]
 [  0 249]]
